In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *
from setting import *

In [ ]:
gene_x_sample = pd.read_table(PATH_DICT["gene_x_sample_file_path"], index_col=0)

target_x_sample = pd.read_table(PATH_DICT["target_x_sample_file_path"], index_col=0)

gmt = ccal.read_gmts(PATH_DICT["gene_set_file_paths"])

gene_set_x_sample = pd.read_table(PATH_DICT["gene_set_x_sample_file_path"], index_col=0)

In [ ]:
for target_name, target_values in target_x_sample.iterrows():

    output_directory_path = "{}/{}".format(
        PATH_DICT["expanded_gene_set_directory_path"], target_name
    )

    ccal.establish_path(output_directory_path, "directory")

    target_values = target_values[target_values != -1]

    score_moe_p_value_fdr__gene = pd.read_table(
        "{}/{}/all.tsv".format(
            PATH_DICT["differentially_expressed_gene_directory_path"], target_name
        ),
        index_col=0,
    )

    score_moe_p_value_fdr__gene_set = pd.read_table(
        "{}/{}/all.tsv".format(
            PATH_DICT["differentially_expressed_gene_set_directory_path"], target_name
        ),
        index_col=0,
    )

    score_moe_p_value_fdr = pd.concat(
        (score_moe_p_value_fdr__gene, score_moe_p_value_fdr__gene_set)
    )

    for gene_set_name in GENE_SET_TO_PEEK:

        features = pd.concat(
            (
                gene_x_sample.reindex(gmt.loc[gene_set_name].dropna()),
                gene_set_x_sample.loc[[gene_set_name]],
            )
        )

        score_moe_p_value_fdr_ = ccal.make_match_panel(
            target_values,
            features,
            score_moe_p_value_fdr=score_moe_p_value_fdr,
            target_ascending=TARGET_ASCENDING,
            extreme_feature_threshold=None,
            target_type=TARGET_TYPE,
            plot_std=PLOT_STD,
            title=gene_set_name,
            file_path_prefix="{}/{}".format(output_directory_path, gene_set_name),
        )